In [1]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [7]:
def delete_data_in_range(ref, start_ts, end_ts):
    """
    Menghapus data dari Firebase Realtime Database dalam rentang timestamp tertentu.
    Versi ini sudah memperbaiki error 'shallow'.
    """
    try:
        print(f"Mencari data untuk dihapus antara {start_ts} dan {end_ts}...")

        # 1. Query untuk mendapatkan semua data dalam rentang waktu yang ditentukan.
        query = ref.order_by_key().start_at(str(start_ts)).end_at(str(end_ts))
        
        # --- PERUBAHAN DI SINI ---
        # Menghapus argumen 'shallow=True' yang menyebabkan error.
        # Kode ini akan mengambil data lengkap (key dan value) di rentang tersebut.
        data_to_delete = query.get()

        # 2. Periksa apakah ada data yang ditemukan
        if not data_to_delete:
            print("Tidak ada data yang ditemukan dalam rentang waktu tersebut.")
            return

        keys_to_delete = list(data_to_delete.keys())
        total_keys = len(keys_to_delete)
        print(f"Ditemukan {total_keys} data yang akan dihapus. Memulai proses penghapusan...")

        # 3. Siapkan dictionary untuk operasi update massal (menghapus dengan menyetel nilai ke None)
        updates = {}
        for key in keys_to_delete:
            updates[key] = None
        
        # Lakukan semua penghapusan dalam satu operasi besar untuk efisiensi
        ref.update(updates)

        print(f"Selesai! Berhasil menghapus {total_keys} data dari '{ref.path}'.")

    except Exception as e:
        print(f"Terjadi error: {e}")


In [3]:
from datetime import datetime
from zoneinfo import ZoneInfo # Direkomendasikan untuk timezone
# Asumsi Anda sudah menginisialisasi firebase_admin dan db
# source_cred = credentials.Certificate("path/to/your/key.json")
# source_app = firebase_admin.initialize_app(source_cred, name='sourceApp', options={ ... })
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
})


In [14]:
# Referensi ke path data yang ingin dihapus
# Konfigurasi database tujuan
dest_ref = db.reference('/auto_weather_stat/id-05/data') # Jangan lupa sesuaikan 'app=dest_app' jika perlu

# --- Tentukan Rentang Waktu Penghapusan ---
# Misalnya, kita ingin menghapus semua data pada tanggal 6 September 2025

# Tentukan zona waktu (sangat penting untuk akurasi)
jakarta_tz = ZoneInfo("Asia/Jakarta")

# Waktu Mulai: DD-MM-YYYY HH:MM:SS
start_dt_str = "02-11-2025 09:14:55"
start_dt_naive = datetime.strptime(start_dt_str, "%d-%m-%Y %H:%M:%S")
start_dt_aware = start_dt_naive.replace(tzinfo=jakarta_tz)
start_timestamp = int(start_dt_aware.timestamp())

# Waktu Selesai DD-MM-YYYY HH:MM:SS
end_dt_str = "02-11-2025 12:21:43"
end_dt_naive = datetime.strptime(end_dt_str, "%d-%m-%Y %H:%M:%S")
end_dt_aware = end_dt_naive.replace(tzinfo=jakarta_tz)
end_timestamp = int(end_dt_aware.timestamp())

print(f"Rentang waktu yang akan dihapus (Unix): {start_timestamp} hingga {end_timestamp}")

# --- Panggil Fungsi untuk Menghapus Data ---
delete_data_in_range(dest_ref, start_timestamp, end_timestamp)

Rentang waktu yang akan dihapus (Unix): 1762049695 hingga 1762060903
Mencari data untuk dihapus antara 1762049695 dan 1762060903...
Ditemukan 21 data yang akan dihapus. Memulai proses penghapusan...
Selesai! Berhasil menghapus 21 data dari '/auto_weather_stat/id-05/data'.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>